In [1]:
# some of these imports might not be needed
# im too lazy to remove them
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np

In [2]:
# define the net
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 32x32 at the start because input is single channel images of
        # size: 32x32
        self.fc1 = nn.Linear(32*32, 16)
        # single hidden layer
        self.fc2 = nn.Linear(16, 16)
        # 3 classes on the output (square / triangle / x shape)
        self.fc3 = nn.Linear(16, 3)
        
    def forward(self, x):
        # x is a flattened version of the single channel image
        display_layer(x)
        x = F.relu(self.fc1(x))
        display_layer(x)
        x = F.relu(self.fc2(x))
        display_layer(x)
        x = F.sigmoid(self.fc3(x))
        display_layer(x.unsqueeze(0), square_image=False)
        return x

net = Net()
# remove these two lines if you dont have CUDA / GPU available
GPU = torch.device("cuda")
net.to(GPU)

Net(
  (fc1): Linear(in_features=1024, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
)

In [4]:
# initian learning rate is the learning rate at the start of the training
# (it changes every epoch)
initlr = 0.001

loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=initlr)

In [5]:
# this is used to load images from a directory into batches later in the dataset class
def load_image(image_path):
    img = Image.open(image_path)
    img = img.convert("L")
    img_array = np.array(img) / 255

    return torch.tensor(img_array, dtype=torch.float32, device=GPU)

In [6]:
# this function takes in an input, output, and network and trains it
def train_nn(question_tensor, answer_tensor, model, loss=loss, optimizer=optimizer):
    optimizer.zero_grad()
    outputs = model(question_tensor)
    loss = loss(outputs, answer_tensor)
    loss.backward()
    optimizer.step()
    return loss.item()

In [7]:
# this is used to later visualize the layers
def display_layer(layer, square_image=True):
    layer = layer.detach().cpu()
    if square_image:
        side_length = int(np.sqrt(len(layer)))
        layer = np.reshape(layer, (side_length, side_length))
        plt.imshow(layer)
        plt.show()
    else:
        plt.imshow(layer)
        plt.show()

In [8]:
# this goes over a test dataset/loader and returns the number of errors the network made
def test_network(net, dataloader):
    errors = 0
    
    for x in enumerate(dataloader):
        if torch.argmax(x[1][1]) != torch.argmax(net(x[1][0])):
            errors += 1
    
    return errors

In [9]:
# this mf takes in a directory and returns a ready dataset with images as inputs, and onehot encoded labels on the other
class ImageDataset():
    def __init__(self, main_dir):
        self.questions = []
        self.answers = []
        
        self.class_folders = os.listdir(main_dir)
        
        for x in tqdm(range(len(self.class_folders))):
            self.current_dir = os.listdir(f"{main_dir}/{self.class_folders[x]}")
            
            for y in self.current_dir:
                self.questions.append(load_image(f"{main_dir}/{self.class_folders[x]}/{y}").flatten())
                self.answers.append(torch.eye(len(self.class_folders), device=GPU)[x])
    
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return self.questions[idx], self.answers[idx]

In [10]:
#                              both assume that the
#                      dataset is in the same dir as this file
#                                       |
#                                       V
train_dataset = ImageDataset(r".\img_dataset\train")
test_dataset = ImageDataset(r".\img_dataset\test")

#                                       the dataset is so small that
#                                       i can fit it whole on my GPU
#                                                    |
#                                                    V
train_loader = DataLoader(train_dataset, batch_size=999, shuffle=True)
#                          the batch size here HAS to remain 1 because if not 
#                       it causes problems with the argmax in the test_net() func
#                                                 |
#                                                 V
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

100%|██████████| 3/3 [00:00<00:00, 96.75it/s]


In [11]:
# you can plot the lists if you want
# mine (from the vid) are somewhere in the repo
# everything is commented so i dont have to wait every time i restart the notebook
loss_list = []
error_list = []
lr_list = []
epochs = 100
lr_decent_rate = 0.005

for y in tqdm(range(epochs)):
    for x in enumerate(train_loader):
        #loss_list.append(train_nn(x[1][0], x[1][1], net))
        #error_list.append(test_network(net, test_loader))
        pass
    
    #lr = initlr * (lr_decent_rate ** (y/epochs))
    #optimizer = optim.Adam(net.parameters(), lr=lr)
    #lr_list.append(lr)

100%|██████████| 100/100 [00:00<00:00, 3702.99it/s]


In [12]:
#torch.save(net.state_dict(), fr"xts_classifier_{100 - test_network(net, test_loader) / len(test_dataset) * 100:.03f}%acc.pth")

In [13]:
state_dict = torch.load(r".\xts_classifier_80.000%acc.pth")
net.load_state_dict(state_dict)

<All keys matched successfully>